<a href="https://colab.research.google.com/github/abhayratti/exoplanet-detection/blob/main/Exoplanet_Detection_Regression_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/cumulative.csv')
data.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [ ]:
archive_info = ['kepler_name', 'kepid', 'kepoi_name', 'koi_disposition', 'koi_pdisposition', 'rowid']
data = data.drop(archive_info, axis=1)

In [ ]:
project_disposition = ['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec']
data = data.drop(project_disposition, axis=1)

In [ ]:
threshold_crossing_events = ['koi_model_snr', 'koi_tce_plnt_num', 'koi_tce_delivname']
data = data.drop(threshold_crossing_events, axis=1)

In [ ]:
kic_parameters = ['ra', 'dec', 'koi_kepmag']
data = data.drop(kic_parameters, axis=1)

In [ ]:
transit_properties = ['koi_period_err1', 'koi_period_err2', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration_err1', 
                      'koi_duration_err2', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq_err1', 'koi_teq_err2', 'koi_insol_err1', 
                      'koi_insol_err2', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad_err1', 'koi_srad_err2']
data = data.drop(transit_properties, axis=1)

In [ ]:
pd.set_option('display.max_columns', 50)

data.dropna(inplace=True)
data.head()

,koi_score,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_slogg,koi_srad
0,1.000,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,5455.0,4.467,0.927
1,0.969,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,5455.0,4.467,0.927
2,0.000,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,5853.0,4.544,0.868
3,0.000,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,5805.0,4.564,0.791
4,1.000,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,6031.0,4.438,1.046


In [ ]:
data.shape

(7995, 12)

In [ ]:
data.columns

Index(['koi_score', 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration',
       'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff',
       'koi_slogg', 'koi_srad'],
      dtype='object')

In [ ]:
features = ['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_srad']
data_features = data[features]

data_features.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_slogg,koi_srad
0,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,5455.0,4.467,0.927
1,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,5455.0,4.467,0.927
2,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,5853.0,4.544,0.868
3,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,5805.0,4.564,0.791
4,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,6031.0,4.438,1.046


In [ ]:
data_target = data['koi_score']

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import numpy as np

In [ ]:
class ExoplanetDataset(torch.utils.data.Dataset):

    def __init__(self, train=True):
        x_train, x_test, y_train, y_test = train_test_split(torch.from_numpy(data_features.values).float(), 
                                                            torch.from_numpy(data_target.values).float(), 
                                                            test_size=0.2, 
                                                            random_state=0)
        
        if train:
            self.x_data, self.y_data = x_train, y_train
        else:
            self.x_data, self.y_data = x_test, y_test

    def __len__(self):
        return self.y_data.shape[0]
    
    def __getitem__(self, i):
        return self.x_data[i], self.y_data[i]

In [ ]:
train_loader = torch.utils.data.DataLoader(ExoplanetDataset(train=True), batch_size=12)
test_loader = torch.utils.data.DataLoader(ExoplanetDataset(train=False), batch_size=12)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
input_size = 11
output_size = 1
hid1_size = 10
hid2_size = 10

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hid1_size)
    self.fc2 = nn.Linear(hid1_size, hid2_size)
    self.fc3 = nn.Linear(hid2_size, output_size)
  
  def forward(self, x):
    x = torch.sigmoid(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))
    x = self.fc3(x)

    return torch.sigmoid(x)

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss(reduction='mean')

In [ ]:
epoch_data = []
epochs = 1000

In [ ]:
for epoch in range(1, epochs + 1):
    valid_loss = 0
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        output_train = model(x)
        loss = loss_fn(output_train.squeeze(1), y)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            output_test = model(x)
            loss = loss_fn(output_test.squeeze(1), y)
            valid_loss += loss.item()
            valid_loss /= len(test_loader)

    if epoch % 100 == 0:
        print(f"Training loss: {loss.item()} \| Valid loss: {valid_loss} \| Accuracy {accuracy}")
    

tensor([[0.0096],
        [0.8721],
        [0.0040]]) tensor([0.0000, 0.4410, 0.0000]) tensor([[ 0.0096, -0.4314,  0.0096],
        [ 0.8721,  0.4311,  0.8721],
        [ 0.0040, -0.4370,  0.0040]])
